# Learning pyswip

In [1]:
import sys; sys.version
#from pyswip import Prolog
import re
from nltk.stem import WordNetLemmatizer

sys.path.append("../python")
from POS_tagger import Tagger

#pl = Prolog()

PROLEXA_PATH = "../prolog/"

## Imports (consults)

In [ ]:
pl.consult(PROLEXA_PATH + "prolexa.pl")

In [21]:
#from pyswip import Variable

def isplural(word):
    wnl = WordNetLemmatizer()
    lemma = wnl.lemmatize(word, 'n')
    plural = True if word is not lemma else False
    return plural

def handle_utterance_str(text) :
    if text[0] != "'" and text[0] != '"' :
        text = f'"{text}"'
    
    text = text.replace('"', '\"')
    text = text.replace("'", '\"')
    
    return "handle_utterance(1,{},Output)".format(text)

def tag(text) :
    tagged_sent, sent, tags = Tagger().tag(text)
    return tags

def update_rules(text):
    tags = tag(text)
    text = text.split(' ')
    start = ''
    end = ''
    f = open(PROLEXA_PATH + "prolexa_grammar.pl", "r")
    lines=f.readlines()
    f.close()
    myiter = iter(lines)
    for idx, line in enumerate(myiter):
        if not text:
            break
        if ('DT' in tags) and re.match(r"determiner\([a-z],X=>B,X=>H,\[\(H:-B\)\]\)(.*)", line):
            start = '--> ['
            end = ']'
            exists = False
            input_word = text[tags.index('DT')]
            for det_idx, det_line in enumerate(lines[idx:]):                                                
                if not(re.match(r"determiner\([a-z],X=>B,X=>H,\[\(H:-B\)\]\)(.*)", det_line)):
                    det_idx = det_idx + idx   
                    if tags:
                        tags.remove('DT')
                    if text:
                        text.remove(input_word)
                    break
                line_word = (det_line.split(start))[1].split(end)[0]  
                if input_word == line_word:                    
                    exists = True
                    if tags:
                        tags.remove('DT')
                    if text:
                        text.remove(input_word)
                    break 

            if not exists:
                if isplural(input_word):
                    new_line = "determiner(p,X=>B,X=>H,[(H:-B)]) --> [{}].\n".format(input_word) 
                else:
                    new_line = "determiner(s,X=>B,X=>H,[(H:-B)]) --> [{}].\n".format(input_word) 
                lines.insert(det_idx, new_line)
                #next(myiter, None)
                    
        if ('NN' in tags) and re.match(r"pred\((.*)[1],\[(.*)\]\)\.", line): 
            start = 'pred('
            end = ', '
            exists = False
            new_line = ''
            input_word = text[tags.index('NN')]
            for noun_idx, noun_line in enumerate(lines[idx:]):
                if not(re.match(r"pred\((.*)[1],\[(.*)\]\)\.", noun_line)):
                    noun_idx = noun_idx + idx   
                    if tags:
                        tags.remove('NN')
                    if text:
                        text.remove(input_word)
                    break
                line_word = (noun_line.split(start))[1].split(end)[0]
                if input_word == line_word:
                    if (re.match(r"pred\((.*)[1](.*)n\/(.*)\]\)\.", noun_line)):
                        exists = True
                        if tags:
                            tags.remove('NN')
                        if text:
                            text.remove(input_word)
                        break
                    else:
                        print("TEST")
                        noun_idx = noun_idx + idx
                        insert_idx = noun_line.index(']).')
                        new_line = noun_line[:insert_idx] + ',n/' + input_word + noun_line[insert_idx:]
                        lines[noun_idx] = new_line
                        exists = True
                        if tags:
                            tags.remove('NN')
                        if text:
                            text.remove(input_word)
                        break

            if not exists:
                if new_line == '':
                    new_line = 'pred(' + input_word + ', 1,[n/' + input_word + ']).\n'
                lines.insert(noun_idx, new_line)
            
                    


    f = open(PROLEXA_PATH + "prolexa_grammar.pl", "w")
    lines = "".join(lines)
    f.write(lines)
    f.close()
            

def escape_and_call_prolexa(text) :
    libPrefix = "prolexa:"
    update_rules(text)
    return ""
    #return pl.query(libPrefix + handle_utterance_str(text))


out = escape_and_call_prolexa('the human is')
list(out)

2020-05-25 17:27:21,622 loading file /home/panjh/.flair/models/en-pos-ontonotes-v0.5.pt
TEST


[]

In [17]:

#Tagger().tag("I have no idea what I'm doing")
#Tagger().tag("Penguins re mrotal")
Tagger().tag("every human is mortal")

2020-05-25 17:25:44,053 loading file /home/panjh/.flair/models/en-pos-ontonotes-v0.5.pt


('every <DT> human <NN> is <VBZ> mortal <JJ>',
 'every human is mortal',
 ['DT', 'NN', 'VBZ', 'JJ'])

In [ ]:
from pyswip import Prolog, Functor, Variable, Query, call

pl = Prolog()
pl.consult("../prolog/prolexa.pl")

In [ ]:
handle_utterance = Functor("handle_utterance", 3)
Output = Variable()
q = Query(handle_utterance(1, "Peter is mortal.", Output), )

while q.nextSolution() :
    print(Output.value)

In [ ]:
q.nextSolution()

In [ ]:
q.nextSolution()

In [ ]:
pl.consult("trains.pl")
STATIONS = [ans["S"] for ans in pl.query("station(S)")]
STATIONS

In [ ]:
pl.assertz("father(michael,john)")
pl.assertz("father(michael,gina)")
pl = Prolog()

for soln in pl.query("father(X,Y)"):
    print(soln["X"], "is the father of", soln["Y"])

# michael is the father of john
# michael is the father of gina

childrenOfMichael = pl.query("father(michael,X)")
list(childrenOfMichael)